---
# Retraining the RNN model on the entire dataset for deployment
---

In this notebook, we will try to build an RNN model that will forecast the daily variability (with seasonal variability) of maximum/minimum temperatures and its variation caused by climate change for the climate weather station located in Downtown Montreal.


### Loading necessary libraries

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import joblib

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Define files

In [4]:
#data_dir = '../data/'
data_dir = 'C:\\Users\\jocl0\\ABF\\Final_Project-main\\data\\'
model_dir = 'C:\\Users\\jocl0\\ABF\\Final_Project-main\\model\\'

#ghg = 'GreenHouse_Gases/processed/'
ghg = 'GreenHouse_Gases\\processed\\'

frequency = 'DLY'

climate_file = 'ECCC\\processed\\daily\\daily_processed.csv'
#climate_file = 'ECCC/processed/daily/daily_processed.csv'
#climate_file = 'ECCC/processed/monthly/monthly_processed.csv'

population_file = 'Population\\processed\\Montreal.csv'
#population_file = 'Population/processed/Montreal.csv'

co2_file = 'co2_1850-2099_SSP2_45.csv'
ch4_file = 'ch4_1850-2099_SSP2_45.csv'

scenario = 'SSP2-45'

### Define variables

**Daily**

In [7]:
Tmax = 'Max Temp (°C)'
Tmin =  'Min Temp (°C)' 
Tmean = 'Mean Temp (°C)'

cols = ['Max Temp (°C)', 'Max Temp Flag', 'Min Temp (°C)', 'Min Temp Flag',
       'Mean Temp (°C)', 'Mean Temp Flag', 'Heat Deg Days (°C)',
       'Heat Deg Days Flag', 'Cool Deg Days (°C)', 'Cool Deg Days Flag',
       'Total Rain (mm)', 'Total Rain Flag', 'Total Snow (cm)',
       'Total Snow Flag', 'Total Precip (mm)', 'Total Precip Flag',
       'Snow on Grnd (cm)', 'Snow on Grnd Flag', 'Dir of Max Gust (10s deg)',
       'Dir of Max Gust Flag', 'Spd of Max Gust (km/h)',
       'Spd of Max Gust Flag']

**Monthly**

### Read data files

In [10]:
co2_df = pd.read_csv(os.path.join(data_dir, ghg, co2_file), index_col=0, parse_dates=True)
co2_df.drop(columns = 'Data Source', inplace=True)
co2_df

,CO2
Year,
1850-01-01,285.200000
1851-01-01,285.100000
1852-01-01,285.000000
1853-01-01,285.000000
1854-01-01,284.900000
...,...
2095-01-01,600.885986
2096-01-01,601.365967
2097-01-01,601.796997


In [11]:
ch4_df = pd.read_csv(os.path.join(data_dir, ghg, ch4_file), index_col=0, parse_dates=True)
ch4_df

,CH4
Year,
1850-01-01,791.600000
1851-01-01,792.859068
1852-01-01,794.156502
1853-01-01,795.493289
1854-01-01,796.859405
...,...
2095-01-01,1698.879883
2096-01-01,1695.429810
2097-01-01,1692.139893


In [12]:
pop_df = pd.read_csv(os.path.join(data_dir, population_file), index_col=0, parse_dates=True)
pop_df

,Population
1801-01-01,9.000000e+03
1802-01-01,9.389547e+03
1803-01-01,9.788026e+03
1804-01-01,1.019544e+04
1805-01-01,1.061293e+04
...,...
2095-01-01,2.258385e+06
2096-01-01,2.249969e+06
2097-01-01,2.240726e+06
2098-01-01,2.230707e+06


In [13]:
wx_df = pd.read_csv(os.path.join(data_dir, climate_file), index_col=0, parse_dates=True)
wx_df

C:\Users\jocl0\AppData\Local\Temp\ipykernel_17796\1851272773.py:1: DtypeWarning: Columns (5,11,13,15,17,19,27,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  wx_df = pd.read_csv(os.path.join(data_dir, climate_file), index_col=0, parse_dates=True)


,Station,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
Date/Time,,,,,,,,,,,,,,,,,,,,,
1891-10-22,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,22,NaN,5.6,...,0.0,T,0.0,T,NaN,NaN,NaN,NaN,NaN,NaN
1891-10-23,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,23,NaN,3.9,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-10-24,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,24,NaN,5.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-10-25,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,25,NaN,3.3,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-10-26,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,26,NaN,10.6,...,0.0,NaN,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-23,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,3,23,NaN,-1.6,...,NaN,NaN,0.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN
2025-03-24,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,3,24,NaN,3.9,...,NaN,NaN,6.7,NaN,9.0,NaN,NaN,NaN,NaN,NaN
2025-03-25,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,3,25,NaN,4.5,...,NaN,NaN,1.1,NaN,11.0,NaN,NaN,NaN,NaN,NaN


### Adding columns of lagged temperature features (yesterday's min/max temperature)

In [14]:
wx_df['tmin_lag1'] = wx_df[Tmin].shift(1)
wx_df['tmax_lag1'] = wx_df[Tmax].shift(1)
wx_df

,Station,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag,tmin_lag1,tmax_lag1
Date/Time,,,,,,,,,,,,,,,,,,,,,
1891-10-22,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,22,NaN,5.6,...,0.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-10-23,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,23,NaN,3.9,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7,5.6
1891-10-24,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,24,NaN,5.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.1,3.9
1891-10-25,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,25,NaN,3.3,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.2,5.0
1891-10-26,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1891,10,26,NaN,10.6,...,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-23,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,3,23,NaN,-1.6,...,0.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,-7.5,10.7
2025-03-24,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,3,24,NaN,3.9,...,6.7,NaN,9.0,NaN,NaN,NaN,NaN,NaN,-9.8,-1.6
2025-03-25,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2025,3,25,NaN,4.5,...,1.1,NaN,11.0,NaN,NaN,NaN,NaN,NaN,-4.6,3.9


### Remove incomplete start and end year of climate dataset

In [16]:
tmp = wx_df.groupby(['Year']).count()

tmp[tmp['Station'] < 365]

,Station,Longitude (x),Latitude (y),Station Name,Climate ID,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,...,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag,tmin_lag1,tmax_lag1
Year,,,,,,,,,,,,,,,,,,,,,
1891,71,71,71,71,71,71,71,0,71,0,...,71,10,0,0,0,0,0,0,70,70
2025,86,86,86,86,86,86,86,0,86,0,...,85,1,86,0,27,0,27,0,86,86


In [17]:
start_wx = wx_df.index.min()
end_wx = wx_df.index.max()
print(type(start_wx), end_wx)

for year in [start_wx, end_wx] :

    nbr_days_year = 366 if pd.Period(year, freq='Y').is_leap_year else 365

    yr = int(year.strftime('%Y'))
    nbr_obs_year = tmp[tmp.index == yr]['Station'].values[0]

    print('*',yr, nbr_obs_year, '*')
    
    if nbr_obs_year != nbr_days_year :
        print(f'Not full year. Removing partial {yr} from dataframe')
        wx_df = wx_df[wx_df['Year'] != yr]

        start_wx = wx_df.index.min()
        end_wx = wx_df.index.max()
    else :
        print(f'Full year. Not removing {yr} from obs dataframe.')
    
wx_df

<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2025-03-27 00:00:00
* 1891 71 *
Not full year. Removing partial 1891 from dataframe
* 2025 86 *
Not full year. Removing partial 2025 from dataframe


,Station,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag,tmin_lag1,tmax_lag1
Date/Time,,,,,,,,,,,,,,,,,,,,,
1892-01-01,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1892,1,1,NaN,-5.6,...,0.0,T,NaN,NaN,NaN,NaN,NaN,NaN,-18.3,-10.6
1892-01-02,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1892,1,2,NaN,6.7,...,11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.7,-5.6
1892-01-03,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1892,1,3,NaN,2.2,...,11.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.1,6.7
1892-01-04,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1892,1,4,NaN,-7.2,...,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.8,2.2
1892-01-05,Montreal (McTavish/McGill),-73.58,45.5,MONTREAL MCGILL,7025280,1892,1,5,NaN,-12.2,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-15.0,-7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2024,12,27,NaN,-6.1,...,0.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,-13.7,-7.9
2024-12-28,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2024,12,28,NaN,-1.7,...,0.2,NaN,11.0,NaN,NaN,NaN,NaN,NaN,-13.2,-6.1
2024-12-29,Montreal (McTavish/McGill),-73.58,45.5,MCTAVISH,7024745,2024,12,29,NaN,7.6,...,5.2,NaN,11.0,NaN,NaN,NaN,NaN,NaN,-9.6,-1.7


### Put the GreenHouse Gases dataframe and population dataframe on the same frequency as the climate data

In [26]:
if frequency == 'DLY':
    new_range = pd.date_range(start = start_wx, end = end_wx, freq="D")
    print(new_range)
    
    co2_df = co2_df.reindex(new_range)
    co2_df = co2_df.resample('D').interpolate()

    ch4_df = ch4_df.reindex(new_range)
    ch4_df = ch4_df.resample('D').interpolate()

    pop_df = pop_df.reindex(new_range)
    pop_df = pop_df.resample('D').interpolate()
    
elif frequency == 'MLY':
    new_range = pd.date_range(start = start_wx, end = end_wx, freq="ME")
    print(new_range)

    co2_df = co2_df.reindex(new_range)
    co2_df = co2_df.resample('ME').interpolate()

    ch4_df = ch4_df.reindex(new_range)
    ch4_df = ch4_df.resample('ME').interpolate()

    pop_df = pop_df.reindex(new_range)
    pop_df = pop_df.resample('ME').interpolate()
else :
    print('Frequency is NOT monthly or daily')

DatetimeIndex(['1892-01-01', '1892-01-02', '1892-01-03', '1892-01-04',
               '1892-01-05', '1892-01-06', '1892-01-07', '1892-01-08',
               '1892-01-09', '1892-01-10',
               ...
               '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25',
               '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', length=48578, freq='D')


In [27]:
co2_df

,CO2
1892-01-01,294.800000
1892-01-02,294.799727
1892-01-03,294.799454
1892-01-04,294.799180
1892-01-05,294.798907
...,...
2024-12-27,424.610000
2024-12-28,424.610000
2024-12-29,424.610000
2024-12-30,424.610000


In [28]:
ch4_df

,CH4
1892-01-01,859.921329
1892-01-02,859.925885
1892-01-03,859.930442
1892-01-04,859.934998
1892-01-05,859.939554
...,...
2024-12-27,1934.802141
2024-12-28,1934.802141
2024-12-29,1934.802141
2024-12-30,1934.802141


In [29]:
pop_df

,Population
1892-01-01,2.609199e+05
1892-01-02,2.609370e+05
1892-01-03,2.609540e+05
1892-01-04,2.609711e+05
1892-01-05,2.609881e+05
...,...
2024-12-27,2.200807e+06
2024-12-28,2.200807e+06
2024-12-29,2.200807e+06
2024-12-30,2.200807e+06


### Put all the data into a single dataframe

In [325]:
full_df = pd.concat([co2_df, ch4_df, pop_df, wx_df[[Tmax,Tmin]], wx_df[['tmin_lag1', 'tmax_lag1']]], axis=1)
full_df

,CO2,CH4,Population,Max Temp (°C),Min Temp (°C),tmin_lag1,tmax_lag1
1892-01-01,294.800000,859.921329,2.609199e+05,-5.6,-11.7,-18.3,-10.6
1892-01-02,294.799727,859.925885,2.609370e+05,6.7,-6.1,-11.7,-5.6
1892-01-03,294.799454,859.930442,2.609540e+05,2.2,-7.8,-6.1,6.7
1892-01-04,294.799180,859.934998,2.609711e+05,-7.2,-15.0,-7.8,2.2
1892-01-05,294.798907,859.939554,2.609881e+05,-12.2,-16.7,-15.0,-7.2
...,...,...,...,...,...,...,...
2024-12-27,424.610000,1934.802141,2.200807e+06,-6.1,-13.2,-13.7,-7.9
2024-12-28,424.610000,1934.802141,2.200807e+06,-1.7,-9.6,-13.2,-6.1
2024-12-29,424.610000,1934.802141,2.200807e+06,7.6,-2.6,-9.6,-1.7
2024-12-30,424.610000,1934.802141,2.200807e+06,10.6,3.3,-2.6,7.6


### Add a day of the year column and the sin curve forcing column

In [135]:
full_df['dayofyear'] = full_df.index.dayofyear
full_df['sin'] = 80 * np.sin((np.pi/(365+31)) * full_df['dayofyear']) -30
full_df

,CO2,CH4,Population,Max Temp (°C),Min Temp (°C),tmin_lag1,tmax_lag1,dayofyear,sin
1892-01-01,294.800000,859.921329,2.609199e+05,-5.6,-11.7,-18.3,-10.6,1,-29.365341
1892-01-02,294.799727,859.925885,2.609370e+05,6.7,-6.1,-11.7,-5.6,2,-28.730723
1892-01-03,294.799454,859.930442,2.609540e+05,2.2,-7.8,-6.1,6.7,3,-28.096184
1892-01-04,294.799180,859.934998,2.609711e+05,-7.2,-15.0,-7.8,2.2,4,-27.461765
1892-01-05,294.798907,859.939554,2.609881e+05,-12.2,-16.7,-15.0,-7.2,5,-26.827506
...,...,...,...,...,...,...,...,...,...
2024-12-27,424.610000,1934.802141,2.200807e+06,-6.1,-13.2,-13.7,-7.9,362,-8.682095
2024-12-28,424.610000,1934.802141,2.200807e+06,-1.7,-9.6,-13.2,-6.1,363,-9.294476
2024-12-29,424.610000,1934.802141,2.200807e+06,7.6,-2.6,-9.6,-1.7,364,-9.908161
2024-12-30,424.610000,1934.802141,2.200807e+06,10.6,3.3,-2.6,7.6,365,-10.523110


### Separate RNN input and output data

In [143]:
cols_in = ['CH4', 'CO2', 'Population', 'tmin_lag1', 'tmax_lag1', 'sin']
cols_out = [Tmax, Tmin]

X_train = full_df[cols_in]
Y_train = full_df[cols_out]

In [ ]:
X_train

### Scale the dataset

In [147]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled_train = scaler_x.fit_transform(X_train)
Y_scaled_train = scaler_y.fit_transform(Y_train)

### TimeSeries Generator

In [153]:
window_size = 31

ts_generator = TimeseriesGenerator(X_scaled_train, 
                                   Y_scaled_train,
                                   length=window_size,
                                   batch_size=64)

ts_generator[0]

(array([[[0.00000000e+00, 8.21665754e-04, 0.00000000e+00, 2.62184874e-01,
          2.79389313e-01, 0.00000000e+00],
         [4.40288378e-06, 8.19420766e-04, 1.13552697e-05, 3.73109244e-01,
          3.55725191e-01, 7.99616772e-03],
         [8.80576755e-06, 8.17175778e-04, 2.27105395e-05, 4.67226891e-01,
          5.43511450e-01, 1.59913289e-02],
         ...,
         [1.23280746e-04, 7.58806079e-04, 3.17947553e-04, 3.27731092e-01,
          3.64885496e-01, 2.21868815e-01],
         [1.27683629e-04, 7.56561091e-04, 3.29302823e-04, 4.11764706e-01,
          3.64885496e-01, 2.29647551e-01],
         [1.32086513e-04, 7.54316102e-04, 3.40658092e-04, 4.11764706e-01,
          3.31297710e-01, 2.37411330e-01]],
 
        [[4.40288378e-06, 8.19420766e-04, 1.13552697e-05, 3.73109244e-01,
          3.55725191e-01, 7.99616772e-03],
         [8.80576755e-06, 8.17175778e-04, 2.27105395e-05, 4.67226891e-01,
          5.43511450e-01, 1.59913289e-02],
         [1.32086513e-05, 8.14930789e-04, 3.406

In [155]:
print(X_scaled_train.shape[0], X_scaled_train.shape[1])
print(Y_scaled_train.shape[1])

47117 6
2


### Load RNN model

In [ ]:
rnn = load_model(os.path.join(model_dir, 'rnn_months_sin_1month_1024k_more.keras'))

In [159]:
rnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1024)              4222976   
                                                                 
 dense_2 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 4,225,026
Trainable params: 4,225,026
Non-trainable params: 0
_________________________________________________________________


### Train the model on entire dataset

In [193]:
rnn.fit(ts_generator, epochs=150)

Epoch 1/50
736/736 [==============================] - 315s 429ms/step - loss: 0.0053
Epoch 2/50
736/736 [==============================] - 340s 462ms/step - loss: 0.0053
Epoch 3/50
736/736 [==============================] - 392s 533ms/step - loss: 0.0053
Epoch 4/50
736/736 [==============================] - 340s 461ms/step - loss: 0.0053
Epoch 5/50
736/736 [==============================] - 368s 500ms/step - loss: 0.0053
Epoch 6/50
736/736 [==============================] - 335s 456ms/step - loss: 0.0053
Epoch 7/50
736/736 [==============================] - 329s 447ms/step - loss: 0.0053
Epoch 8/50
736/736 [==============================] - 332s 451ms/step - loss: 0.0053
Epoch 9/50
736/736 [==============================] - 331s 450ms/step - loss: 0.0053
Epoch 10/50
736/736 [==============================] - 335s 456ms/step - loss: 0.0053
Epoch 11/50
736/736 [==============================] - 313s 424ms/step - loss: 0.0053
Epoch 12/50
736/736 [==============================] - 310s 422

### Save model and scalers to use in deployment

In [250]:
rnn.save(os.path.join(model_dir, 'rnn_months_sin_1month_1024k_more_entire.keras'))

In [253]:
joblib.dump(scaler_x, os.path.join(model_dir, 'scaler_x.joblib'))
joblib.dump(scaler_y, os.path.join(model_dir, 'scaler_y.joblib'))

['C:\\Users\\jocl0\\ABF\\Final_Project-main\\model\\scaler_y.joblib']